## AlexNet
在2012年, 由 [Alex Krizhevsky](https://www.cs.toronto.edu/~kriz/), [Ilya Sutskever](http://www.cs.toronto.edu/~ilya/), [Geoffrey Hinton](http://www.cs.toronto.edu/~hinton/)提出了一种使用卷积神经网络的方法, 以 [0.85](http://image-net.org/challenges/LSVRC/2012/results.html#abstract) 的`top-5`正确率一举获得当年分类比赛的冠军, 超越使用传统方法的第二名10个百分点, 震惊了当时的学术界, 从此开启了人工智能领域的新篇章.

下面复现一次`AlexNet`, 首先来看它的网络结构

<img src="https://ws4.sinaimg.cn/large/006tKfTcgy1fodv2ppd85j307l0n13yk.jpg">

可以看出`AlexNet`就是几个卷积池化堆叠后连接几个全连接层, 下面就让我们来尝试仿照这个结构来解决[cifar10](https://www.cs.toronto.edu/~kriz/cifar.html)分类问题.

In [1]:
import torch
from torch import nn
import numpy as np
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

依照上面的结构，我们可以定义 AlexNet

In [2]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 第一层是 5x5 的卷积, 输入的 channels 是 3，输出的 channels 是 64, 步长是 1, 没有 padding
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 5), 
            nn.ReLU(True))
        
        # 第二层是 3x3 的池化, 步长是 2, 没有 padding
        self.max_pool1 = nn.MaxPool2d(3, 2)
        
        # 第三层是 5x5 的卷积, 输入的 channels 是 64，输出的 channels 是 64, 步长是 1, 没有 padding
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, 5, 1),
            nn.ReLU(True))

        # 第四层是 3x3 的池化，步长是 2，没有 padding
        self.max_pool2 = nn.MaxPool2d(3, 2)
        
        # 第五层是全连接层，输入是 1204，输出是 384
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 384),
            nn.ReLU(True))
        
        # 第六层是全连接层，输入是 384，输出是 192
        self.fc2 = nn.Sequential(
            nn.Linear(384, 192),
            nn.ReLU(True))
        
        # 第七层是全连接层，输入是 192，输出是 10
        self.fc3 = nn.Linear(192, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        
        # 将矩阵拉平
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [3]:
alexnet = AlexNet()

打印一下网络的结构

In [4]:
alexnet

AlexNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (max_pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (max_pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=1024, out_features=384, bias=True)
    (1): ReLU(inplace)
  )
  (fc2): Sequential(
    (0): Linear(in_features=384, out_features=192, bias=True)
    (1): ReLU(inplace)
  )
  (fc3): Linear(in_features=192, out_features=10, bias=True)
)

我们验证一下网络结构是否正确，输入一张 32 x 32 的图片，看看输出

In [5]:
# 定义输入为 (1, 3, 32, 32)
input_demo = Variable(torch.zeros(1, 3, 32, 32))
output_demo = alexnet(input_demo)
print(output_demo.shape)

torch.Size([1, 10])


In [6]:
from utils import train

def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    x = x.transpose((2, 0, 1)) # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x
     
train_set = CIFAR10('./data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = AlexNet().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [7]:
train(net, train_data, test_data, 20, optimizer, criterion)

F:\Notebook\pytorch_learning\utils.py:52: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
F:\Notebook\pytorch_learning\utils.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 1.737648, Train Acc: 0.360354, Valid Loss: 1.818357, Valid Acc: 0.332278, Time 00:00:12
Epoch 1. Train Loss: 1.271463, Train Acc: 0.543558, Valid Loss: 1.802944, Valid Acc: 0.450356, Time 00:00:13
Epoch 2. Train Loss: 1.030234, Train Acc: 0.637308, Valid Loss: 1.035772, Valid Acc: 0.639043, Time 00:00:13
Epoch 3. Train Loss: 0.871309, Train Acc: 0.695013, Valid Loss: 1.180607, Valid Acc: 0.613924, Time 00:00:13
Epoch 4. Train Loss: 0.754152, Train Acc: 0.736773, Valid Loss: 0.948707, Valid Acc: 0.667820, Time 00:00:13
Epoch 5. Train Loss: 0.659849, Train Acc: 0.770301, Valid Loss: 0.987987, Valid Acc: 0.678995, Time 00:00:13
Epoch 6. Train Loss: 0.581576, Train Acc: 0.796875, Valid Loss: 1.116473, Valid Acc: 0.663469, Time 00:00:13
Epoch 7. Train Loss: 0.505174, Train Acc: 0.821072, Valid Loss: 0.870209, Valid Acc: 0.725376, Time 00:00:13
Epoch 8. Train Loss: 0.435892, Train Acc: 0.847806, Valid Loss: 0.938800, Valid Acc: 0.722903, Time 00:00:13
Epoch 9. Train Loss

可以看到，训练 20 次，AlxeNet 能够在 cifar 10 上取得 70% 左右的测试集准确率